In [2]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets
from sklearn.decomposition import PCA
import numpy as np
import scipy.io as sio
#from PIL import Image
import random

#Defition for loading contents:

mat_contents = sio.loadmat('ex3data1.mat')
#print(mat_contents) # Prints the metadata of the '.mat' file.
XContents = mat_contents['X']
YContents = mat_contents['y']
print("Type of XContents : ",type(XContents))
#print(XContents)
#print(YContents)
#print(len(XContents) , len(XContents[0]))
np.savetxt('ex3data1.txt', XContents)


# Functions used for logistic regression:

J_of_theta=[0.0]*5000
no_of_iterations = 50
alpha = 0.01

def regularization(theta,lambdaValue):
    theta_excluding_zeroth_index = np.zeros( shape = (1,len(theta)) )
    theta_excluding_zeroth_index[0] = 0.0
    for i in range(1,len(theta)):
        theta_excluding_zeroth_index[0][i] = (theta[i][0]*theta[i][0])
    sum=0
    for i in range(0,len(theta)):
        sum = (sum + theta_excluding_zeroth_index[0][i])
        
    return sum,theta_excluding_zeroth_index

def calculate_h_of_theta(theta,X):
    h_of_theta = X.dot(theta)
    #print(h_of_theta)
    return h_of_theta

def sigmoid(h_of_theta):
    return 1.0/(1+np.exp(-1.0 * h_of_theta))
    
def costFunction(theta,X,y,lambdaValue):
    h_of_theta = calculate_h_of_theta(theta,X)
    m = len(h_of_theta)
    i=0
    J=0
    for i in range(0,m):
        J += ( (-float(1.0)/float(m)) * ( ( float(y[i][0]) * np.log(sigmoid(float(h_of_theta[i][0]))) ) + \
            ( (1.0-float(y[i][0])) * np.log(1.0 - sigmoid(float(h_of_theta[i][0])))) ) )
        
        regularizationSum,theta_excluding_zeroth_index = regularization(theta,lambdaValue)
        
        J += (lambdaValue/(2*m))*regularizationSum

    return J,theta_excluding_zeroth_index
    

def calculateDelta(theta,X,y,theta_excluding_zeroth_index,lambdaValue):
    h_of_theta = calculate_h_of_theta(theta,X)
    m = len(h_of_theta)
    i=0
    for i in range(0,len(h_of_theta)):
        h_of_theta[i] = sigmoid(h_of_theta[i])
    
    #print("Dimensions returned by calculateDelta : ",len(theta_excluding_zeroth_index),len(theta_excluding_zeroth_index[0]))
    return (1/m)*(np.transpose(X).dot((h_of_theta - y)))  +  ( (lambdaValue/m) * np.transpose(theta_excluding_zeroth_index))
    
def updateTheta(theta,X,y,lambdaValue):
    print("Update Theta() called!!")
    print("updateTheta() Dimensions of theta : ",len(theta),len(theta[0]))
    i=0
    while(i<50):
        J_of_theta[i],theta_excluding_zeroth_index = costFunction(theta,X,y,lambdaValue)
        #print(J_of_theta[i])
        
        #Breaking condition:
        if( (i>0) and ( (J_of_theta[i] - J_of_theta[i-1]) > 0 ) ):
            break
        
        theta = theta - ( alpha * calculateDelta(theta,X,y,theta_excluding_zeroth_index,lambdaValue))
        i=i+1
        
    print("End of loop")
    print("Minimised Cost : ",J_of_theta[i-1])
    return theta
    

#Python code to display images of random indices:

def displayRandomDataSet(X):
    L = 0
    R = 4999
    numberOfSamples = 10
    indicesList = random.sample(range(L,R),numberOfSamples)
    image = np.zeros((20,20))
    k=0
    i=0
    j=0
    for listIndex in range(0,len(indicesList)-1):
        row = indicesList[listIndex]
        k=0
        i=0
        j=0
        for i in range(0,20):
            for j in range(0,20):
                image[j][i] = X[row][k]
                k = k+1
                j = j+1
            i = i+1
        plt.imshow(image,cmap = 'gray')
        plt.show()
        listIndex = listIndex + 1


def displayGivenDataSet(X,index):
    image = np.zeros((20,20))
    k=0
    print(type(index),index)
    row = int(index)
    
    for i in range(0,20):
        for j in range(0,20):
            image[j][i] = X[row][k]
            k = k+1
            j = j+1
        i = i+1
    plt.imshow(image,cmap = 'gray')
    plt.show()
    



def oneVsAll(n,numberOfLabels,X,y,lambdaValue):
    
    allTheta = np.zeros( shape = (numberOfLabels,n) )
    i=1
    for i in range(1,numberOfLabels+1):
        
        initialTheta = np.zeros(shape=(n,1))
        tempY = np.zeros(shape = (1,len(X)))
        
        for j in range (0,len(X)):
            if(y[j] == i):
                tempY[0][j] = 1
            else:
                tempY[0][j] = 0
            j = j + 1 
            
        tempY = tempY.transpose()
        theta = updateTheta(initialTheta,X,tempY,lambdaValue)
        print("Dimensions of theta : " , len(theta),len(theta[0]))
        for k in range(0,len(theta)):
            allTheta[i-1][k] = theta[k]
            k = k + 1
        
        i = i + 1
        
    return allTheta

    

# Appending 1 at each row of X:
print("Type of XContents[0] : ",type(XContents[0]))
X = np.zeros(shape = ( len(XContents) , len(XContents[0])+1 ))
y = []
i=0
for i in range(0,len(XContents)):
    y.append(YContents[i][0])
#print(y)

i=0
for i in range(0,len(XContents)):
    X[i][0] = 1.0
    j = 0
    for j in range(0,len(XContents[0])-1):
        X[i][j+1] = XContents[i][j]
        j = j + 1
    i = i + 1

#displayDataSet(X)
numberOfLabels = 10
lambdaValue = 0.1
print(len(X[0]))
allTheta = oneVsAll(len(X[0]),numberOfLabels,X,y,lambdaValue)

#print(tempY[0])
np.savetxt('test.txt', allTheta,delimiter=",")


print("Iterations Complete")



Type of XContents :  <class 'numpy.ndarray'>
Type of XContents[0] :  <class 'numpy.ndarray'>
401
Update Theta() called!!
updateTheta() Dimensions of theta :  401 1
End of loop
Minimised Cost :  0.281922089198
Dimensions of theta :  401 1
Update Theta() called!!
updateTheta() Dimensions of theta :  401 1
End of loop
Minimised Cost :  0.355078187428
Dimensions of theta :  401 1
Update Theta() called!!
updateTheta() Dimensions of theta :  401 1
End of loop
Minimised Cost :  0.355097288497
Dimensions of theta :  401 1
Update Theta() called!!
updateTheta() Dimensions of theta :  401 1
End of loop
Minimised Cost :  0.330060346828
Dimensions of theta :  401 1
Update Theta() called!!
updateTheta() Dimensions of theta :  401 1
End of loop
Minimised Cost :  0.345218675021
Dimensions of theta :  401 1
Update Theta() called!!
updateTheta() Dimensions of theta :  401 1
End of loop
Minimised Cost :  0.342356811125
Dimensions of theta :  401 1
Update Theta() called!!
updateTheta() Dimensions of theta